# Downloading MODIS NDVI

This product: https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD43A4#description

## Load packages

In [ ]:
import os
import ee
import geemap
import wxee
import warnings
import xarray as xr
import pandas as pd
import numpy as np
import eemont
from odc.geo.xr import assign_crs

## Authenticate GEE    

In [ ]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Analysis parameters


In [ ]:
# product = 'MODIS/MCD43A4_006_NDVI'
product = 'MODIS/061/MCD43A4'
bands = ['NDVI']
start_date = '3/1/2000'
end_date = '12/31/2022'
output_crs = "EPSG:3577"
resolution = 5000

# ~Aus region
region = ee.Geometry.Polygon([[
            [112,-43.0],
            [155,-43.0],
            [155,-9.0],
            [112,-9.0],
            [112,-43.0]]])


In [ ]:
def filterqa(image):
    mask1 = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band1").eq(0)
    mask2 = image.select("BRDF_Albedo_Band_Mandatory_Quality_Band2").eq(0)
    return image.updateMask(mask1).updateMask(mask2) 

def calc_ndvi(image):
    ndvi = image.normalizedDifference(['nir','red']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='M')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")

    ss = s+pd.Timedelta(14, 'd')
    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = (ee.ImageCollection(product) 
      .filterDate(s,e) 
      .map(lambda image: image.clip(region))
      .map(filterqa)
      .select(["Nadir_Reflectance_Band1", "Nadir_Reflectance_Band2"], ["red", "nir"])
      .map(lambda image: image.multiply(0.0001))
      .map(calc_ndvi)
      .reduce(ee.Reducer.median())
      .select('NDVI_median')
         )
 
    export = ee.batch.Export.image.toDrive(ds,
        description='MODIS_NDVI_median'+'_'+ss.strftime('%Y-%m-%d'),
        folder='MODIS_NDVI_median',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()
    
    i+=1

In [ ]:
export.status()

In [ ]:
# Using basemap in geemap
Map = geemap.Map(center=[-35.2041, 149.2721], zoom=4)

ndvi_palette=['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

ndviVis = {'min': 0, 'max': 1, 'palette': ndvi_palette}

Map.addLayer(ds, ndviVis)
Map

## Stitch together geotiffs

In [ ]:
import os
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs

In [ ]:
base='/g/data/os22/chad_tmp/climate-carbon-interactions/data/MODIS_NDVI_median/'
name= 'NDVI_median'
dates_start = '2000-03-01'
dates_end = '2022-12-31'


In [ ]:
files = [base+i for i in os.listdir(base) if i.endswith('.tif')]
files.sort()

time = (pd.date_range(dates_start, dates_end, freq='MS') + pd.DateOffset(days=14)).to_numpy()
time = list(time)

print(len(files), print(len(time)))

In [ ]:
i=0
arrs=[]
for f, t in zip(files, time):
    print(" {:02}/{:02}\r".format(i + 1, len(files)), end="")
    
    ds = xr.open_dataarray(f).squeeze().drop('band').astype('float32')
    ds = ds.expand_dims(time=[t])
    ds = ds.rename(name)
    del ds.attrs['long_name']
    del ds.attrs['AREA_OR_POINT']

    ds.attrs['nodata']=np.nan
    ds = ds.drop('spatial_ref')
    ds = assign_crs(ds, crs='epsg:3577')
    arrs.append(ds)

    i+=1 
    

In [ ]:
ds = xr.concat(arrs, dim='time').sortby('time')

In [ ]:
mask = xr.open_dataarray('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/MODIS_NDVI_5km_monthly_2001_2022.nc').mean('time')
mask = ~np.isnan(mask)

In [ ]:
ds = ds.where(mask)

In [ ]:
ds.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/MODIS_'+name+'_5km_monthly_2000.nc')